In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import string
import json
import re
from sklearn.utils import shuffle
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import sklearn
from sklearn.metrics import confusion_matrix
####
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model

from sklearn.linear_model import LogisticRegression
import sklearn.linear_model as linear_model
import sklearn.model_selection as model_selection

from sklearn import svm
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB

In [2]:
# open training dataset and parse recipes

with open('train.json') as cooking_file:  
    
    data = json.load(cooking_file)
    

In [3]:
#COUNTVECTORIZER
#removing punctuations and spaces before fixing the data and keep them all in a dict  (italian, mexican)

punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

def gather_recipes(recipes, cuisine_country):
    
    data_list = []
    
    for _object in recipes:
        
        if _object.get('cuisine') == cuisine_country:
            
            ingredients = _object.get('ingredients') 

            _id = _object.get('id')
            
            for i in ingredients:
                
                _dict = dict()
                ingr = i.replace(' ', '').lower()
                    
                for ch in ingr: 
                    
                    if ch in punctuations: 
                        
                        ingr = ingr.replace(ch, '')  
                
                _dict['ingredients'] = ingr                
                _dict['id'] = _id
                data_list.append(_dict)
                
    return data_list


clean_italian_cuisine_data = gather_recipes(data, 'italian')
clean_mexican_cuisine_data = gather_recipes(data, 'mexican')

clean_filipino_cuisine_data = gather_recipes(data, 'filipino')
clean_british_cuisine_data = gather_recipes(data, 'british')
clean_moroccan_cuisine_data = gather_recipes(data, 'moroccan')

In [4]:
#creating dfs  (italian, mexican)

italian_df = DataFrame(clean_italian_cuisine_data)

mexican_df = DataFrame(clean_mexican_cuisine_data)


#creating dfs (filipino, british, moroccan )

filipino_df = DataFrame(clean_filipino_cuisine_data) 

british_df = DataFrame(clean_british_cuisine_data)

moroccan_df = DataFrame(clean_moroccan_cuisine_data)


In [5]:
#fixing data for CountVectorizer (italian, mexican)

it_unique_ids = []
it_ingredients = []
it_grouped = italian_df.groupby('id')

for ids, ing in it_grouped:
    
    it_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    it_ingredients.append(row)

    
mex_unique_ids = []
mex_ingredients = []
mex_grouped = mexican_df.groupby('id')

for ids, ing in mex_grouped:
    
    mex_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    mex_ingredients.append(row)
    
#fixing data for CountVectorizer (moroccan, filipino, british)

moroccan_unique_ids = []
moroccan_ingredients = []

moroccan_grouped = moroccan_df.groupby('id')

for ids, ing in moroccan_grouped:
    
    moroccan_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    moroccan_ingredients.append(row)


filipino_unique_ids = []
filipino_ingredients = []

filipino_grouped = filipino_df.groupby('id')

for ids, ing in filipino_grouped:
    
    filipino_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    filipino_ingredients.append(row)


british_unique_ids = []
british_ingredients = []

british_grouped = british_df.groupby('id')

for ids, ing in british_grouped:
    
    british_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    british_ingredients.append(row)

In [6]:
#using CountVectorizer  (italian, mexican) and adding one column Cuisine

it_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
it_array = it_vectorizer.fit_transform(it_ingredients)
it_array = it_array.toarray()
it_df = DataFrame(it_array, columns = it_vectorizer.get_feature_names(), index = it_unique_ids)
it_df['Cuisine'] = 0 #italian
it_df.to_csv('it_cleandata.csv', index_label = 'ID')

mex_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
mex_array = mex_vectorizer.fit_transform(mex_ingredients)
mex_array = mex_array.toarray()
mex_df = DataFrame(mex_array, columns = mex_vectorizer.get_feature_names(), index = mex_unique_ids)
mex_df['Cuisine'] = 1 #mexican
mex_df.to_csv('mex_cleandata.csv', index_label = 'ID')

#using CountVectorizer (moroccan, filipino, british)

moroccan_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
moroccan_array = moroccan_vectorizer.fit_transform(moroccan_ingredients)
moroccan_array = moroccan_array.toarray()
moroccan_df = DataFrame(moroccan_array, columns = moroccan_vectorizer.get_feature_names(), index = moroccan_unique_ids)
moroccan_df['Cuisine'] = 0 #moroccan
moroccan_df.to_csv('moroccan_cleandata.csv', index_label = 'ID')

filipino_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
filipino_array = filipino_vectorizer.fit_transform(filipino_ingredients)
filipino_array = filipino_array.toarray()
filipino_df = DataFrame(filipino_array, columns = filipino_vectorizer.get_feature_names(), index = filipino_unique_ids)
filipino_df['Cuisine'] = 1 #filipino
filipino_df.to_csv('filipino_cleandata.csv', index_label = 'ID')

british_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
british_array = british_vectorizer.fit_transform(british_ingredients)
british_array = british_array.toarray()
british_df = DataFrame(british_array, columns = british_vectorizer.get_feature_names(), index = british_unique_ids)
british_df['Cuisine'] = 2 #british
british_df.to_csv('british_cleandata.csv', index_label = 'ID')

In [7]:
#putting mexican and italian in one DataFrame and shuffle

df = pd.DataFrame(it_df)
df = df.append(mex_df, sort = False)
df.fillna(0, inplace = True)
df = df.astype('int64')

df = shuffle(df)

#putting moroccan, filipino, british in one DataFrame and shuffle

df_ = pd.DataFrame(moroccan_df)
df_ = df_.append(filipino_df, sort = False)
df_ = df_.append(british_df, sort = False)
df_.fillna(0, inplace = True)
df_ = df_.astype('int64')

df_ = shuffle(df_)

In [8]:
#TFIDFVECTORIZER
#removing punctuations before fixing the data and keep them all in a dict  (italian, mexican)

def gather_recipes(recipes, cuisine_country):
    
    data_list = []
    
    for _object in recipes:
        
        if _object.get('cuisine') == cuisine_country:
            
            ingredients = _object.get('ingredients') 
            
            _id = _object.get('id')
            
            for i in ingredients:
                
                _dict = dict()
                ingr = i.lower()
                    
                for ch in ingr: 
                    
                    if ch in punctuations: 
                        
                        ingr = ingr.replace(ch, '')  
                
                _dict['ingredients'] = ingr                
                _dict['id'] = _id
                data_list.append(_dict)
                
    return data_list

clean2_italian_cuisine_data = gather_recipes(data, 'italian')
clean2_mexican_cuisine_data = gather_recipes(data, 'mexican')

In [9]:
#creating dfs2  (italian, mexican)

italian2_df = DataFrame(clean2_italian_cuisine_data)

mexican2_df = DataFrame(clean2_mexican_cuisine_data)

In [10]:
#fixing data for TfidfVectorizer (italian, mexican)

it2_unique_ids = []
it2_ingredients = []
it2_grouped = italian2_df.groupby('id')

for ids, ing in it2_grouped:
    
    it2_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    it2_ingredients.append(row)

    
mex2_unique_ids = []
mex2_ingredients = []
mex2_grouped = mexican2_df.groupby('id')

for ids, ing in mex2_grouped:
    
    mex2_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    mex2_ingredients.append(row)


In [11]:
#using TfidfVectorizer  (italian, mexican) and adding one column Cuisine

it2_vectorizer = TfidfVectorizer(analyzer = 'word', binary = True)
it2_array = it2_vectorizer.fit_transform(it2_ingredients)
it2_array = it2_array.toarray()
it2_df = DataFrame(it2_array, columns = it2_vectorizer.get_feature_names(), index = it2_unique_ids)
it2_df['Cuisine'] = 0 #italian
it2_df.to_csv('it2_cleandata.csv', index_label = 'ID')

mex2_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
mex2_array = mex2_vectorizer.fit_transform(mex2_ingredients)
mex2_array = mex2_array.toarray()
mex2_df = DataFrame(mex2_array, columns = mex2_vectorizer.get_feature_names(), index = mex2_unique_ids)
mex2_df['Cuisine'] = 1 #mexican
mex2_df.to_csv('mex2_cleandata.csv', index_label = 'ID')

In [12]:
#putting mexican and italian in one DataFrame and shuffle

df2 = pd.DataFrame(it2_df)
df2 = df2.append(mex2_df, sort = False)
df2.fillna(0, inplace = True)

df2 = shuffle(df2)

In [13]:
# Multiple Logistic Regression 
lr_m = linear_model.LogisticRegression()

# Support Vector Machines
svm_m = svm.LinearSVC(random_state=0)
#Decision Trees
dtree = tree.DecisionTreeClassifier()

#k-NN
knn = KNeighborsClassifier(n_neighbors = 3)

# Naive Bayes
gnb = BernoulliNB(binarize = None)

In [14]:
print('************ CountVectorizer ITALIAN-MEXICAN ************')
X = df.loc[:, df.columns != 'Cuisine'].values  #all columns with values except Cuisine column
print(X.shape)

Y = df['Cuisine'] #just the ids in a list

************ CountVectorizer ITALIAN-MEXICAN ************
(14276, 3976)


- 1. ITALIAN MEXICAN CUISINES CLASSIFIERS (COUNTVECTORIZER)

In [15]:
s = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
scores_ = model_selection.cross_validate(lr_m, X, Y, scoring = s, cv = 5)                                 
print('****Logistic Regression mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(lr_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Logistic Regression mean scores****

scores: {'fit_time': array([2.02100396, 2.34072614, 2.16998649, 2.11153007, 2.2689724 ]), 'score_time': array([0.0307045 , 0.02660918, 0.02673864, 0.0268352 , 0.02677441]), 'test_accuracy': array([0.96918768, 0.97548161, 0.9642732 , 0.96462347, 0.96287215]), 'test_precision_weighted': array([0.96938465, 0.9756432 , 0.96428349, 0.96474671, 0.96310721]), 'test_recall_weighted': array([0.96918768, 0.97548161, 0.9642732 , 0.96462347, 0.96287215]), 'test_f1_weighted': array([0.96914685, 0.97545321, 0.96425491, 0.96458425, 0.9628166 ])}

mean score test accuracy: 0.9672876225796798

mean score test precision weighted: 0.9674330532458463

mean score test recall weighted: 0.9672876225796798

mean score test f1-measure weighted: 0.9672511628603668
Confusion Matrix: [[7677  161]
 [ 306 6132]]


In [16]:
scores_ = model_selection.cross_validate(svm_m, X, Y, scoring = s, cv = 5)                                 
print('****SVM mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(svm_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****SVM mean scores****

scores: {'fit_time': array([0.50101781, 0.50441217, 0.50834394, 0.5069232 , 0.51732135]), 'score_time': array([0.02665138, 0.02882767, 0.02671981, 0.03081417, 0.02783012]), 'test_accuracy': array([0.96743697, 0.96812609, 0.96112084, 0.96322242, 0.96392294]), 'test_precision_weighted': array([0.96745523, 0.96812976, 0.96111477, 0.96323596, 0.96395864]), 'test_recall_weighted': array([0.96743697, 0.96812609, 0.96112084, 0.96322242, 0.96392294]), 'test_f1_weighted': array([0.96741912, 0.9681134 , 0.96111674, 0.96320231, 0.96389761])}

mean score test accuracy: 0.9647658538021163

mean score test precision weighted: 0.9647788729597322

mean score test recall weighted: 0.9647658538021163

mean score test f1-measure weighted: 0.9647498335735223
Confusion Matrix: [[7618  220]
 [ 283 6155]]


In [17]:
scores_ = model_selection.cross_validate(dtree, X, Y, scoring = s, cv = 5)                                 
print('****Decision-Tree mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(dtree, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Decision-Tree mean scores****

scores: {'fit_time': array([16.02788186, 15.03037262, 14.49209857, 16.44369054, 10.01070046]), 'score_time': array([0.0299387 , 0.02936292, 0.02980471, 0.03082323, 0.02949643]), 'test_accuracy': array([0.94502801, 0.94255692, 0.93870403, 0.94360771, 0.94010508]), 'test_precision_weighted': array([0.94506545, 0.94277854, 0.93874452, 0.94359041, 0.94010116]), 'test_recall_weighted': array([0.94502801, 0.94255692, 0.93870403, 0.94360771, 0.94010508]), 'test_f1_weighted': array([0.94504101, 0.94260074, 0.93871859, 0.94358959, 0.940103  ])}

mean score test accuracy: 0.9420003483004411

mean score test precision weighted: 0.9420560141078564

mean score test recall weighted: 0.9420003483004411

mean score test f1-measure weighted: 0.9420105851231474
Confusion Matrix: [[7437  401]
 [ 430 6008]]


In [18]:
scores_ = model_selection.cross_validate(knn, X, Y, scoring = s, cv = 5)                                 
print('****k-NN mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(knn, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****k-NN mean scores****

scores: {'fit_time': array([4.13730311, 4.23047996, 4.1768105 , 4.16403461, 4.18332767]), 'score_time': array([171.64351153, 170.7505672 , 170.47388315, 170.26147747,
       172.90324879]), 'test_accuracy': array([0.8855042 , 0.87565674, 0.87390543, 0.88406305, 0.87915937]), 'test_precision_weighted': array([0.88643296, 0.87715688, 0.87478601, 0.88750753, 0.88248912]), 'test_recall_weighted': array([0.8855042 , 0.87565674, 0.87390543, 0.88406305, 0.87915937]), 'test_f1_weighted': array([0.88500882, 0.87494841, 0.87333561, 0.88308359, 0.87813849])}

mean score test accuracy: 0.8796577580244007

mean score test precision weighted: 0.8816744984267748

mean score test recall weighted: 0.8796577580244007

mean score test f1-measure weighted: 0.8789029839045679
Confusion Matrix: [[7286  552]
 [1166 5272]]


In [19]:
scores_ = model_selection.cross_validate(gnb, X, Y, scoring = s, cv = 5)                                 
print('****Naive-Bayes mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(gnb, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Naive-Bayes mean scores****

scores: {'fit_time': array([0.46606064, 0.46728015, 0.46936846, 0.46967053, 0.4687984 ]), 'score_time': array([0.03312802, 0.02978611, 0.03024936, 0.02994943, 0.02968359]), 'test_accuracy': array([0.96533613, 0.9709282 , 0.96182137, 0.96252189, 0.9642732 ]), 'test_precision_weighted': array([0.9656972 , 0.9713518 , 0.9620097 , 0.96303278, 0.96477105]), 'test_recall_weighted': array([0.96533613, 0.9709282 , 0.96182137, 0.96252189, 0.9642732 ]), 'test_f1_weighted': array([0.96527317, 0.97087248, 0.96176892, 0.96243988, 0.96419674])}

mean score test accuracy: 0.9649761585895303

mean score test precision weighted: 0.9653725064592347

mean score test recall weighted: 0.9649761585895303

mean score test f1-measure weighted: 0.9649102350754177
Confusion Matrix: [[7704  134]
 [ 366 6072]]


In [20]:
print('************ TfidfVectorizer ITALIAN-MEXICAN ************')
X = df2.loc[:, df2.columns != 'Cuisine'].values  #all columns with values except Cuisine column
print(X.shape)

Y = df2['Cuisine'] #just the ids in a list

************ TfidfVectorizer ITALIAN-MEXICAN ************
(14276, 2006)


- 2. ITALIAN MEXICAN CUISINES CLASSIFIERS (TFIDFVECTORIZER)

In [21]:
s = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
scores_ = model_selection.cross_validate(lr_m, X, Y, scoring = s, cv = 5)                                 
print('****Logistic Regression mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(lr_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Logistic Regression mean scores****

scores: {'fit_time': array([0.49528098, 0.49032998, 0.50628304, 0.52625608, 0.4933126 ]), 'score_time': array([0.00989985, 0.00967813, 0.00979519, 0.00994539, 0.0099411 ]), 'test_accuracy': array([0.9947479 , 0.99299475, 0.99369527, 0.99369527, 0.99334501]), 'test_precision_weighted': array([0.99479767, 0.99308297, 0.99376683, 0.99376687, 0.99342473]), 'test_recall_weighted': array([0.9947479 , 0.99299475, 0.99369527, 0.99369527, 0.99334501]), 'test_f1_weighted': array([0.99474502, 0.99298951, 0.99369106, 0.99369109, 0.99334034])}

mean score test accuracy: 0.9936956393765912

mean score test precision weighted: 0.9937678141684019

mean score test recall weighted: 0.9936956393765912

mean score test f1-measure weighted: 0.9936914043718813
Confusion Matrix: [[7838    0]
 [  90 6348]]


In [22]:
scores_ = model_selection.cross_validate(svm_m, X, Y, scoring = s, cv = 5)                                 
print('****SVM mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(svm_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****SVM mean scores****

scores: {'fit_time': array([0.19249678, 0.19439864, 0.19309926, 0.19654226, 0.20102763]), 'score_time': array([0.00987649, 0.00977659, 0.01011825, 0.00979686, 0.00986075]), 'test_accuracy': array([0.99719888, 0.99509632, 0.99579685, 0.99614711, 0.99614711]), 'test_precision_weighted': array([0.9972131 , 0.99513972, 0.99582877, 0.99617397, 0.99617397]), 'test_recall_weighted': array([0.99719888, 0.99509632, 0.99579685, 0.99614711, 0.99614711]), 'test_f1_weighted': array([0.99719808, 0.99509381, 0.99579502, 0.99614559, 0.99614559])}

mean score test accuracy: 0.9960772540189456

mean score test precision weighted: 0.9961059045720708

mean score test recall weighted: 0.9960772540189456

mean score test f1-measure weighted: 0.996075615953585
Confusion Matrix: [[7838    0]
 [  56 6382]]


In [23]:
scores_ = model_selection.cross_validate(dtree, X, Y, scoring = s, cv = 5)                                 
print('****Decision-Tree mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(dtree, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Decision-Tree mean scores****

scores: {'fit_time': array([2.97214127, 2.61173844, 2.96488023, 3.02945685, 2.89817476]), 'score_time': array([0.01481056, 0.01428556, 0.01474571, 0.0151732 , 0.01443458]), 'test_accuracy': array([0.99894958, 0.99789842, 0.99894921, 0.99894921, 0.99754816]), 'test_precision_weighted': array([0.99895159, 0.99790643, 0.99895122, 0.99894939, 0.99755907]), 'test_recall_weighted': array([0.99894958, 0.99789842, 0.99894921, 0.99894921, 0.99754816]), 'test_f1_weighted': array([0.99894947, 0.99789798, 0.9989491 , 0.99894918, 0.99754755])}

mean score test accuracy: 0.9984589177177

mean score test precision weighted: 0.9984635381530838

mean score test recall weighted: 0.9984589177177

mean score test f1-measure weighted: 0.9984586547933947
Confusion Matrix: [[7836    2]
 [  25 6413]]


In [24]:
scores_ = model_selection.cross_validate(knn, X, Y, scoring = s, cv = 5)                                 
print('****k-NN mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(knn, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****k-NN mean scores****

scores: {'fit_time': array([2.29788756, 2.30485845, 2.2978003 , 2.316818  , 2.30216026]), 'score_time': array([87.95050287, 87.1386826 , 86.84163833, 86.89153743, 86.71721411]), 'test_accuracy': array([0.94292717, 0.93380035, 0.9295972 , 0.93309982, 0.93380035]), 'test_precision_weighted': array([0.94830145, 0.94092143, 0.93759661, 0.94036827, 0.94092548]), 'test_recall_weighted': array([0.94292717, 0.93380035, 0.9295972 , 0.93309982, 0.93380035]), 'test_f1_weighted': array([0.94240947, 0.93305881, 0.92873585, 0.93234251, 0.93306214])}

mean score test accuracy: 0.9346449788321634

mean score test precision weighted: 0.9416226487588105

mean score test recall weighted: 0.9346449788321634

mean score test f1-measure weighted: 0.9339217542204354
Confusion Matrix: [[7838    0]
 [ 933 5505]]


In [25]:
scores_ = model_selection.cross_validate(gnb, X, Y, scoring = s, cv = 5)                                 
print('****Naive-Bayes mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(gnb, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Naive-Bayes mean scores****
scores: {'fit_time': array([0.10244465, 0.09605718, 0.09662342, 0.09556246, 0.09568977]), 'score_time': array([0.01146913, 0.0106802 , 0.01060176, 0.01051474, 0.01059294]), 'test_accuracy': array([0.97969188, 0.97478109, 0.97443082, 0.97688266, 0.97618214]), 'test_precision_weighted': array([0.98041627, 0.97588826, 0.97556827, 0.97781698, 0.97717273]), 'test_recall_weighted': array([0.97969188, 0.97478109, 0.97443082, 0.97688266, 0.97618214]), 'test_f1_weighted': array([0.97964237, 0.97470123, 0.97434849, 0.97681713, 0.97611215])}

mean score test accuracy: 0.9763937168562696

mean score test precision weighted: 0.9773725021265147

mean score test recall weighted: 0.9763937168562696

mean score test f1-measure weighted: 0.9763242753106687
Confusion Matrix: [[7838    0]
 [ 337 6101]]


- 3. MOROCCAN FILIPINO BRITISH CUISINES CLASSIFIERS (COUNTVECTORIZER)

In [26]:
print('********* CountVectorizer MOROCCAN, FILIPINO, BRITISH *********')
X = df_.loc[:, df_.columns != 'Cuisine'].values  #all columns with values except Cuisine column
print(X.shape)

Y = df_['Cuisine'] #just the ids in a list

********* CountVectorizer MOROCCAN, FILIPINO, BRITISH *********
(2380, 2074)


In [27]:
s = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
scores_ = model_selection.cross_validate(lr_m, X, Y, scoring = s, cv = 5)                                 
print('****Logistic Regression mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(lr_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Logistic Regression mean scores****

scores: {'fit_time': array([0.24299026, 0.2705543 , 0.26976728, 0.25782275, 0.26740623]), 'score_time': array([0.00483322, 0.00489259, 0.00492525, 0.004879  , 0.00511003]), 'test_accuracy': array([0.92647059, 0.93697479, 0.92647059, 0.94117647, 0.93697479]), 'test_precision_weighted': array([0.92641729, 0.9396368 , 0.92873015, 0.94198833, 0.93815808]), 'test_recall_weighted': array([0.92647059, 0.93697479, 0.92647059, 0.94117647, 0.93697479]), 'test_f1_weighted': array([0.92642564, 0.93717634, 0.92685848, 0.94122886, 0.93713157])}

mean score test accuracy: 0.9336134453781513

mean score test precision weighted: 0.9349861304549023

mean score test recall weighted: 0.9336134453781513

mean score test f1-measure weighted: 0.9337641784580353
Confusion Matrix: [[775  12  34]
 [ 12 689  54]
 [ 22  24 758]]


In [28]:
scores_ = model_selection.cross_validate(svm_m, X, Y, scoring = s, cv = 5)                                 
print('****SVM mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(svm_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****SVM mean scores****

scores: {'fit_time': array([0.0528028 , 0.05172682, 0.05302501, 0.05210161, 0.05257726]), 'score_time': array([0.00499988, 0.00503659, 0.00487518, 0.00492406, 0.00488973]), 'test_accuracy': array([0.93277311, 0.94117647, 0.92436975, 0.93907563, 0.94327731]), 'test_precision_weighted': array([0.93320769, 0.94221632, 0.92498936, 0.93910264, 0.94342959]), 'test_recall_weighted': array([0.93277311, 0.94117647, 0.92436975, 0.93907563, 0.94327731]), 'test_f1_weighted': array([0.9325179 , 0.94129261, 0.92451957, 0.93903347, 0.94326932])}

mean score test accuracy: 0.9361344537815126

mean score test precision weighted: 0.9365891203224862

mean score test recall weighted: 0.9361344537815126

mean score test f1-measure weighted: 0.936126573299434
Confusion Matrix: [[784  14  23]
 [ 13 698  44]
 [ 22  36 746]]


In [29]:
scores_ = model_selection.cross_validate(dtree, X, Y, scoring = s, cv = 5)                                 
print('****Decision Tree mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(dtree, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Decision Tree mean scores****

scores: {'fit_time': array([0.18636179, 0.15774131, 0.18395233, 0.15329409, 0.16762614]), 'score_time': array([0.00529981, 0.00519848, 0.00518775, 0.00517392, 0.00530577]), 'test_accuracy': array([0.78361345, 0.83823529, 0.80462185, 0.84453782, 0.83403361]), 'test_precision_weighted': array([0.78345522, 0.8395491 , 0.80658583, 0.84462755, 0.83636306]), 'test_recall_weighted': array([0.78361345, 0.83823529, 0.80462185, 0.84453782, 0.83403361]), 'test_f1_weighted': array([0.78328016, 0.83856903, 0.80517769, 0.84411797, 0.83451473])}

mean score test accuracy: 0.8210084033613446

mean score test precision weighted: 0.8221161532678906

mean score test recall weighted: 0.8210084033613446

mean score test f1-measure weighted: 0.8211319166844484
Confusion Matrix: [[703  41  77]
 [ 25 629 101]
 [ 79  82 643]]


In [30]:
scores_ = model_selection.cross_validate(knn, X, Y, scoring = s, cv = 5)                                 
print('****k-NN mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(knn, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****k-NN mean scores****

scores: {'fit_time': array([0.23235416, 0.22900748, 0.22746539, 0.22920609, 0.22975874]), 'score_time': array([2.3134234 , 2.30817127, 2.30964065, 2.31931782, 2.32099175]), 'test_accuracy': array([0.79621849, 0.81722689, 0.82983193, 0.80672269, 0.77731092]), 'test_precision_weighted': array([0.80564666, 0.81865657, 0.83312099, 0.80779577, 0.77848578]), 'test_recall_weighted': array([0.79621849, 0.81722689, 0.82983193, 0.80672269, 0.77731092]), 'test_f1_weighted': array([0.79614516, 0.81754068, 0.83046698, 0.80689481, 0.77727065])}

mean score test accuracy: 0.8054621848739496

mean score test precision weighted: 0.8087411529670405

mean score test recall weighted: 0.8054621848739496

mean score test f1-measure weighted: 0.8056636559935397
Confusion Matrix: [[656 105  60]
 [ 44 631  80]
 [ 82  92 630]]


In [31]:
scores_ = model_selection.cross_validate(gnb, X, Y, scoring = s, cv = 5)                                 
print('****Naive-Bayes mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(gnb, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Naive-Bayes mean scores****

scores: {'fit_time': array([0.04219437, 0.04296708, 0.0433774 , 0.04321694, 0.0430541 ]), 'score_time': array([0.0056386 , 0.0050571 , 0.00505447, 0.00512314, 0.00504565]), 'test_accuracy': array([0.89285714, 0.92226891, 0.91596639, 0.90336134, 0.90546218]), 'test_precision_weighted': array([0.89424351, 0.92944995, 0.92116582, 0.91200426, 0.90939058]), 'test_recall_weighted': array([0.89285714, 0.92226891, 0.91596639, 0.90336134, 0.90546218]), 'test_f1_weighted': array([0.89320523, 0.92274086, 0.91650951, 0.90489788, 0.90614631])}

mean score test accuracy: 0.9079831932773109

mean score test precision weighted: 0.9132508246764672

mean score test recall weighted: 0.9079831932773109

mean score test f1-measure weighted: 0.9086999601267728
Confusion Matrix: [[737   9  75]
 [ 14 668  73]
 [ 20  28 756]]
